In [2]:
# Getting all the libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

We have 2 targets, on one hand target B, which says if someone is going to do a dontaiton or not, and target D which says that people who is going to do a donation, how much is going to done.

# 1. Prediction of target B

In [3]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

Numerical, categorical and target are the files totally clean, without nan values.

In [4]:
numerical = pd.read_csv('files_for_lab/numerical.csv')
categorical = pd.read_csv('files_for_lab/categorical.csv')
targets = pd.read_csv('files_for_lab/target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()  # The data is inbalnce

0    90569
1     4843
Name: TARGET_B, dtype: int64

To make the data balance.

In [5]:
category_0 = data[data['TARGET_B']==0].sample(len(data[data['TARGET_B']==1]))

category_1 = data[data['TARGET_B']== 1 ]
data = pd.concat([category_0, category_1], axis = 0)
data = data.sample(frac =1)    # In order to mix all the raws
data = data.reset_index(drop=True)

In [6]:
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categorcalX = X.select_dtypes(object)

In [7]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categorcalX)
encoded_categorical = encoder.transform(categorcalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

X = pd.concat([numericalX, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.6276458440887971
0.5743034055727554


In [10]:
from sklearn.model_selection import cross_val_score  # For cross validation
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

0.5766616654163541


# 2. Prediction of target D

In [19]:
data = category_1.sample(frac =1)    # Taking all raws where target B is 1, so means that they are doing a donation.
data = data.reset_index(drop=True)

In [20]:
y = data['TARGET_D']
X = data.drop(['TARGET_D'], axis = 1)

numericalX = X.select_dtypes(np.number)
categorcalX = X.select_dtypes(object)

In [21]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categorcalX)
encoded_categorical = encoder.transform(categorcalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

X = pd.concat([numericalX, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

X_train = X_train.drop(['TARGET_B'], axis = 1)
X_test = X_test.drop(['TARGET_B'], axis = 1)

In [29]:
from sklearn.tree import DecisionTreeRegressor
model1 = DecisionTreeRegressor()
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
from sklearn.neighbors import KNeighborsRegressor
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    scores[model_names[i]] = mean_score
    i = i+1

In [25]:
scores

{'Regression Tree': -0.24686911659021482,
 'Linear Regression': 0.1757682662936193,
 'KNN': -0.23655682893419372}